In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [46]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [47]:
##Preprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [48]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [49]:
#Encode categorical variables
labelEncoderGender=LabelEncoder()
data['Gender']=labelEncoderGender.fit_transform(data['Gender'])

In [50]:
##Onehot encode 'Geography' column
onehotEncoderGeo=OneHotEncoder(handle_unknown='ignore')
geoEncoded=onehotEncoderGeo.fit_transform(data[['Geography']]).toarray()
geoEncoded



array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [51]:
geoEncodeddf=pd.DataFrame(geoEncoded,columns=onehotEncoderGeo.get_feature_names_out(['Geography']))
geoEncodeddf

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [52]:
#Combine onehot encoded values with original data
data=pd.concat([data.drop('Geography',axis=1),geoEncodeddf],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [53]:
#Split the data into features and target
X=data.drop('EstimatedSalary',axis=1) #independent variables
y=data['EstimatedSalary'] #dependent variable

In [54]:
#Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [55]:
#Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [56]:
#Save the encoders and scaler for later use
with open('labelEncoderGenderRegression.pkl','wb') as file:
    pickle.dump(labelEncoderGender,file)
with open('onehotEncoderGeoRegression.pkl','wb') as file:
    pickle.dump(onehotEncoderGeo,file)
with open('scalerRegression.pkl','wb')as file:
    pickle.dump(scaler,file)

In [57]:
## ANN Regression Problem Statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [58]:
#Build the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)#Output layer for regression
])

##Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 64)                832       
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
##Setup the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [61]:
##Setup Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [62]:
## Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 7s 12ms/step - loss: 100395.4062 - mae: 100395.4062 - val_loss: 98586.1953 - val_mae: 98586.1953
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 99856.7656 - mae: 99856.7656 - val_loss: 97449.5078 - val_mae: 97449.5078
Epoch 3/100
250/250 [==============================] - 2s 10ms/step - loss: 97800.4141 - mae: 97800.4141 - val_loss: 94370.9766 - val_mae: 94370.9766
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 93586.2891 - mae: 93586.2891 - val_loss: 89024.1875 - val_mae: 89024.1875
Epoch 5/100
250/250 [==============================] - 3s 11ms/step - loss: 87219.8594 - mae: 87219.8594 - val_loss: 81822.0859 - val_mae: 81822.0859
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 79336.3984 - mae: 79336.3984 - val_loss: 73760.0547 - val_mae: 73760.0547
Epoch 7/100
250/250 [==============================] - 2s 10ms/step - loss: 71026.9453 - mae: 71026

In [63]:
model.save('modelRegression.h5')

d:\AIPractice\ANNClassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [69]:
##Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [75]:
%tensorboard --logdir logs/fit20251204-142849

Reusing TensorBoard on port 6008 (pid 16924), started 0:00:10 ago. (Use '!kill 16924' to kill it.)

In [76]:
##Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test Mean Absolute Error :{test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50263.4258 - mae: 50263.4258
Test Mean Absolute Error :50263.42578125


In [77]:
model.save('regressionModel.h5')

d:\AIPractice\ANNClassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
